# How AutoDL Works

# Table of contents

- [Introduction](#Introduction)
- [Implementation in arcgis.learn](#Implementation)
    - [Imports](#imports)
    - [Prepare data](#prepare)
- [Train networks using AutoDL](#AutoDL)
    - [AutoDL Training modes](#ModesAutoDL)
- [Supported methods in AutoDL](#MethodsAutoDL)
    - [Supported Classification Models](#scm)
    - [Supported Detection Models](#sdm)
    - [fit](#fit)
    - [Score](#score)
    - [Show Results](#results)
    - [MIOU](#miou)
    - [Average Precision Score](#aps)
    
- [What is ImageryModel](#ImageryModel)
    - [Load the model](#imLoad)
    - [Learning rate](#imLR)
    - [Train the model](#imFit)
    - [Save the model](#imSave)

# Introduction
<a id='Introduction'></a> 

This guide explains the steps for training and evaluation of multiple network architectures supported by `arcgis.learn` API.
The `arcgis.learn` API currently supports more than 30 deep learning networks to solve different problems, more specifically for object detetcion task only, API provides 4 deep learning networks along with MMDetection class that in turn supports more than twenty object detection networks. Similarly for pixel classification task API provides support for 11 deep learning networks in which MMSegmentation class supports more than twenty pixel classification networks.

To train a deep learning network using the `arcgis.learn` API, one must follow the complete pipeline that involves data preprocessing, network selection, hyper parameter tuning and network selection/evaluation based on the performance of the network. This can sometime become a difficult task for a user to iteratively run all the networks to compare the performance and selecting which network works best with the given data. 

AutoDL class automatically trains all the supported network with the given data within the provided time limit and comes up with a performance tally of all the networks which will provide a fair idea of the best performing network, the AutoDL class also saves all the networks during the training process which can be used later for fine tuning to enhance the network performance.


### AutoDL supported Networks

**Object Detection**

- SingleShotDetector
- RetinaNet
- FasterRCNN
- YOLOv3
- MMDetection:
    - atss
    - carafe
    - cascade_rcnn
    - cascade_rpn
    - dcn

**Pixel Classification**

- DeepLab
- UnetClassifier
- PSPNetClassifier
    - ann
    - apcnet
    - ccnet
    - cgnet
    - deeplabv3


# Implementation in `arcgis.learn`
<a id='Implementation'></a> 

Let's see how AutoDL class works with `arcgis.learn`

## Imports
<a id='imports'></a>

In [ ]:
from arcgis.learn import AutoDL, prepare_data, ImageryModel

## Prepare data
<a id='prepare'></a>

Prepare data for AutoDL class using prepare_data() in `arcgis.learn`

In [ ]:
data = prepare_data("path_to_data_folder", batch_size=2)

# Train networks using AutoDL
<a id='AutoDL'></a>

AutoDL class accepts the following paramters:

- `data` (Required Paramter): Returned data object from `prepare_data` function in the previous step.
    
- `total_time_limit` (Optional parameter):
    The total time limit in hours for AutoDL to train and evaluate the networks. This parameter becomes important when time is the main constraint to the user. The AutoDL class calculates the number of chips that can be processed in the given time from the prepared databunch.
    
    
- `mode` (Optional Parameter): Can be "basic" or "advanced".

    - _basic_ : To to be used when the user wants to train all selected networks.
    - _advanced_ : To be used when the user additionally wants to tune hyper parameters of two best performing models from basic mode.
    

- `networks` (Optional Parameter):
    The list of models that will be used in the training process. If the user does not provides the parameter value the AutoDL class selects all the supported networks, however user can select one or multiple networks by passing the network name in a list.

    - _Supported Object Detection models_
        - SingleShotDetector, RetinaNet, FasterRCNN, YOLOv3, MMDetection
    - _Supported Object Detection models_
        - DeepLab, UnetClassifier, PSPNetClassifier, MMSegmentation
        

- `verbose` (Optional Parameter): Optional Boolean.
    To be used to display logs while training the networks. This parameter displays the progress with time and becomes important in case of any failure, user can use the logs to check which, when and why network training failed.

### AutoDL Training modes
<a id='ModesAutoDL'></a>

- **Basic**
    - In this mode we iterate through all the supported networks exactly once with the default backbone, train it with the passed data and calculate the network performance. At the end of each iteration the function will save the model to the disk. Maximum number of epochs to train each network is 20, however if the remaining time left to process the network is less than than the expected time, the program will automatically calculate the maximum number of epochs to train the network.
    
    
- **Advanced**
    - To be used when the user wants to tune hyper parameters of two best performing networks from basic mode.This mode will divide the total time into two halfs. In the first half it works as basic mode where it will iterate through all the supported networks exactly once. In the second half it checks for two best performing networks. The program then trains the selected networks with different supported backbones. At the end of each iteration the function will save the model to the disk. Maximum number of epochs to train each network is 20, however if the remaining time left to process the network is less than the expected time(minimum time required to train the network) , the program will automatically calculate the number of epochs to train the network.
 

In [ ]:
dl = AutoDL(data, total_time_limit=5,verbose=True, mode="advanced")

When AutoDL class is initialized, it calculates the number of images that can be processed in the given time and the time required to process the entire data. The output of the cell above can be used to analyze and update the `total_time_limit` and `networks` parameters while initializing the class.

**Here is an example of the output**

- Given time to process the dataset is: 5.0 hours
- Number of images that can be processed in the given time: 290
- Time required to process the entire dataset of 3000 images is 52 hours

It explains how many images can be processed to train all the selected network in the selected mode within the given time and also provides an estimate of time that AutoDL will take to train all the selected network with the entire dataset.

# Supported methods in AutoDL
<a id='MethodsAutoDL'></a>

### Supported Classification Models
<a id='scm'></a>

In [ ]:
dl.supported_classification_models()

The output of this function will be a list of supported pixel classification models by `AutoDL` class

### Supported Detection Models
<a id='sdm'></a>

In [ ]:
dl.supported_detection_models()

The output of this function will be a list of supported object detection models by AutoDL class

### Fit
<a id='fit'></a>

The `fit` method will be used to train all the selected networks automatically within the provided time limit

In [ ]:
dl.fit()

### Score
<a id='score'></a>

This method will return a dataframe that will show the model's accuracy with train/valid loss, dice(for pixel classification), learning rate used to train the model, train time and backbone.

In [ ]:
dl.score()

### Show Results 
<a id='results'></a>

This method will display the results for the best performing model

In [ ]:
dl.show_results()

### MIOU 
<a id='miou'></a>

MIOU is mean of intersection over union, this method calculates mean IOU on the validation set for each class. This function is only supported by pixel classification models.

In [ ]:
dl.mIOU()

### Average Precision Score 
<a id='aps'></a>

This method computes the average of the precision on the validation set for each class. This function is only supported by object detection models.

In [ ]:
dl.average_precision_score()

# What is ImageryModel
<a id='ImageryModel'></a>

Once the best performing network is identified it can be further fine tuned using the `ImageryModel` class, this class supports methods that can be used to load, fine-tune and save the model for further use.

In [ ]:
im = ImageryModel()

### Load the model
<a id='imLoad'></a>

- `load` method is used to load a saved model from the disk using `AutoDL` class, it accepts the following parameters:
    - path: Path to the ESRI Model Definition (.emd) file
    - data: Returned data object from prepare_data function

In [ ]:
im.load("path_to_emd_file", data)

### Learning rate Load the model
<a id='imLR'></a>

`lr_find` runs the Learning Rate Finder. Helps in choosing the optimum learning rate for training the model.

In [ ]:
im.lr_find()

### Train the model
<a id='imfit'></a>

The loaded model can be trained further using the fit method. This method trains the model for the specified number of epochs while using the specified learning rates.

In [ ]:
im.fit(10)

### Save the model 
<a id='imSave'></a>

This method saves the model weights, creates an Esri Model Definition and Deep Learning Package zip for deployment to Image Server or ArcGIS Pro.

In [ ]:
im.save("path_to_save_model")

## Conclusion 

This guide explains how AutoDL class can be used to automate multiple deeplearning models supported by `arcgis.learn` API. For every step in the workflow, we define a function and talk about its usage. This guide can be a starting point for developers to train and evaluate multiple `arcgis.learn` supported model's performace.

For more information about the API, please go to the <a href="https://developers.arcgis.com/python/api-reference/arcgis.learn.toc.html#autodl">API reference</a>